In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

model = AzureChatOpenAI(model="gpt-4o",
                        api_version="2025-04-01-preview",
                        temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요. 제 이름은 안민재입니다."),
    AIMessage("안녕하세요. 안민재님. 반가워요."),
    HumanMessage("제 이름이 뭐에요?")
]

ai_message = model.invoke(messages)

print(ai_message.content)

In [ ]:
# Streaming

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕!"),
]
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

In [ ]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
                                      다음 요리의 레시피를 생각해주세요.
                                      요리명: {dish}
                                      """)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)

In [ ]:
# ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
        ("ai", "{recipe}"), #("assistant", "") 도 가능
    ]
)
prompt_value = prompt.invoke({"dish": "카레", "recipe": "카레 레시피"})
pprint(prompt_value.messages)

In [ ]:
# Messages Placeholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요, 저는 안민재입니다.!"),
            AIMessage(content="안녕하세요, 안민재님. 반가워요."),
        ],
        "input": "제 이름이 뭐에요?",
    }
)

pprint(prompt_value.messages)

In [ ]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

In [ ]:
# Output Parsers
# PydanticOutputParser

from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")



from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "사용자가 입력한 요리의 레시피를 생각해 주세요. \n\n"
         "{format_instructions}"
         ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})

print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

In [ ]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

ai_message = model.invoke(prompt_value)
print(ai_message.content)


In [ ]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)


In [ ]:
# StrOutputParser
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요, 저는 AI 어시턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

In [ ]:
# LCEL (LangChain Expression Language)
# 연쇄적으로 연결하고 싶은 경우
# Prompt template 을 채우고, 그 결과를 Chat model에 제공한 후 그 결과를 Python 객체로 변환

# prompt와 model 연결
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}")
    ]
)

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

chain = prompt | model

ai_message = chain.invoke({"dish": "카레"})
print(ai_message.content)


In [ ]:
# StrOutputParser를 연결에 추가
# prompt, model, StrOutputParser 연결은 LCE의 가장 기본적인 형태
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "카레"})
print(output)


In [ ]:
# Pydantic Output Parser를 사용한 연결
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(descriptions="ingredients of the dish")
    steps: list[str] = Field(descriptions="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
propmt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요. \n\n {format_instructions}"
        ),
        ("human", "{dish}")
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

chain = prompt_with_format_instructions | model | output_parser

recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)


In [ ]:
# with_structured_output
# with_structured_output은 기본적으로 Function calling을 사용하여 json 형식의 데이터를 출력
# Function calling은 실제로 함수를 호출하지 않고, json 형식의 데이터를 확실하게 출력하는 목적으로 자주 사용
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}")
    ]
)

model = AzureChatOpenAI(
    model="gpt-4o",
    api_version="2025-04-01-preview",
    temperature=0
)

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['카레 가루', '감자', '당근', '양파', '고기', '물', '소금', '후추'] steps=['감자, 당근, 양파를 적당한 크기로 썬다.', '고기를 준비하여 적당한 크기로 자른다.', '냄비에 기름을 두르고 고기를 볶는다.', '고기가 익으면 썰어둔 채소를 넣고 함께 볶는다.', '물과 카레 가루를 넣고 잘 섞는다.', '중불에서 끓이며 소금과 후추로 간을 맞춘다.', '재료가 모두 익고 국물이 걸쭉해지면 불을 끈다.', '밥과 함께 카레를 접시에 담아낸다.']


In [ ]:
# RAG: 검색증강이라고 단순히 생각하는 것이 아니라 프롬프트에 문맥을 추가하는 방법을 고려
